<a href="https://colab.research.google.com/github/meishihna/GITHUB-COLAB/blob/main/LINE_Notify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**工具**

####定義包

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Stock/'

In [ ]:
!pip install FinMind
!pip install --upgrade mplfinance
!pip install twstock
!pip install lineTool

In [3]:
import pandas as pd
# from Data import getDataYF,getDataFM
import mplfinance as mpf
import matplotlib.pyplot as plt
import os
import numpy as np
import twstock
import lineTool

In [ ]:
import yfinance as yf
import FinMind
from FinMind.data import DataLoader

datapath = "data"
FM = DataLoader()

from Data import getDataFM,getDataFM1,getFMShareHolder,ShareHolderWeekly,getPriceAndShareHolder,getTSEInstitutionalInvestors,getFMInstitutionalInvestors
from Data import InstInvestorsDaily,getPriceAndInstInvest_TSE,getPriceAndInstInvest_FM,getTSEMarginTrading,getFMMarginTrading
from Data import getTSEShortSales,getFMShortSales,getFMPriceAndMarginTrade,getFMPriceAndShortSales,getTSEMonthRevenue,getTSEPriceAndRevenue
from BackTest import ChartCandle,ChartTrade,Performance

def get_company_name(company_code):
    # 將用戶輸入的公司代碼轉換為字串
    company_code = str(company_code)

    stock_info = twstock.codes[company_code]
    if stock_info is not None:
        return f"{stock_info.name}"
    else:
        return f"找不到公司代碼 {company_code} 對應的公司名稱"

####TALIB

In [14]:
!pip install ta-lib-bin > log.txt

In [15]:
import talib
from talib import SMA as func_sma
from talib.abstract import SMA as abs_sma
from talib.abstract import EMA

####LINE Notify

In [18]:
# Line 推播
token = "lppCbwUyaxN2tIBRz1R1gHd5vKpYCve4tyd5TTI501I" #我
# token="PsJ1IPfpwwAsZmQb0oi5U9Xgz3ZsLxIBtEqMyQ5dhS8" #00919股友社
# token ="TcKCmadWHAS7bE9AXdiRPVmyWY7d1wO4lmfc5TRJ2EZ" #梅家
# token = "B2PULu4O1B6YgkWEobko8IqFDkz5CFwnk4bXHUmaUYS"
def line_print(msg):
    print(msg)
    try :
        lineTool.lineNotify(token, msg)
    except:
        print('line notify 失效')
        pass

###**策略**

---



###**MACD策略**

In [26]:
from talib.abstract import MACD

import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getDataFM1(prod,st,en)

# 計算MACD
data=data.join(MACD(data,40,120,60))

# 初始部位
position=0
signal = 1
# 開始回測
for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'high']
    c_close=data.loc[c_time,'close']
    c_macd=data.loc[c_time,'macdhist']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
        if c_macd >0  :
            position = 1
            order_i=i
            order_time=n_time
            order_price=n_open
            signal = 3

        else:
            signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_macd < -0.005 :
            position = 0
            cover_time=n_time
            cover_price=n_open
            signal = 4
        else:
            signal = 2

strategy_name='MACD策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2330


2024-01-26 04:42:07.703 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2330



 MACD策略 
 2330 台積電 
 2024-01-26 
維持進場


###**均線突破策略**




In [17]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data = getDataFM1(prod,st,en)
# EMA(data,timeperiod=120)
data['ema'] = EMA(data,timeperiod=120)

movestoploss = 0.05

position = 0
trade = pd.DataFrame()

for i in range(data.shape[0]-1):
  c_time = data.index[i]
  c_high = data.loc[c_time,'high']
  c_close = data.loc[c_time,'close']
  c_ema = data.loc[c_time,'ema']

  n_time = data.index[i+1]
  n_open = data.loc[n_time,'open']

  if position == 0 :
    if c_close > c_ema*1.01 :
      position = 1
      signal = 3
    else:
      signal = 1
      # print(c_time,'觸發進場訊號 隔日進場', order_time,'進場價', order_price, '進場', order_unit,'單位')

  elif position == 1 :
    if c_close < c_ema * 0.995:
      position = 0
      signal = 4
    else:
      signal = 2

strategy_name='突破均線策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2331


2024-01-26 04:50:10.768 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2331



 突破均線策略 
 2331 【精英】 
 2024-01-26 
維持進場


###**RSI策略**

In [11]:
from talib.abstract import RSI

prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getDataFM1(prod,st,en)

# 計算相對強弱指標
data['rsi1']=RSI(data,timeperiod=120)
data['rsi2']=RSI(data,timeperiod=150)

# 初始部位
position=0
signal = 0
# 開始回測
for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'high']
    c_close=data.loc[c_time,'close']
    c_rsi1=data.loc[c_time,'rsi1']
    c_rsi2=data.loc[c_time,'rsi2']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
        if c_rsi1 > c_rsi2 :
            position = 1
            signal = 3
        else:
            signal = 1
    # 出場程序
    elif position == 1 :
        # 出場邏輯
        if c_rsi1 < c_rsi2  * 0.999 :
            position = 0
            signal = 4
        else:
            signal = 2

result = get_company_name(prod)
strategy_name='RSI策略'
if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2331


2024-01-25 11:45:56.980 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2331



 RSI策略 
 2331 精英 
 2024-01-25 
維持進場


###**強勢回檔策略**

In [37]:
from talib.abstract import RSI
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getDataFM1(prod,st,en)

# 計算相對強弱指標 以及 買超 賣超
data['rsi']=RSI(data,timeperiod=10)
over_buy=80
over_sell=40

# 初始部位
position=0
signal = 0
rsi_min,rsi_min_time=100,0
# 開始回測
for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'high']
    c_close=data.loc[c_time,'close']
    c_rsi=data.loc[c_time,'rsi']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'open']

    # 進場程序
    if position ==0  :
        if c_rsi < over_sell :
            # 如果當前 rsi 等於最小值 則變動
            if rsi_min > c_rsi:
                rsi_min = c_rsi
                rsi_min_time = i
                continue # 直接換隔天
        # 判斷今天在最低點近三日 RSI反彈10
        if i<=rsi_min_time+3 and c_rsi>rsi_min+10:
            rsi_min = 100
            position = 1
            signal = 3
        else:
            signal = 1

    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_rsi > over_buy :
            position = 0
            signal = 4
        else:
            signal = 2

strategy_name='強勢回檔策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))


請輸入要查詢的股票代碼:2331


2024-01-25 09:35:55.240 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2331



 強勢回檔策略 
 2331 精英 
 2024-01-25 
維持空手


###**ATR濾網交易策略**

In [30]:
from talib.abstract import EMA,ATR
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getDataFM1(prod,st,en)

# 計算指數移動平均線
data['ema']=EMA(data,timeperiod=80)
data['atr1']=ATR(data,timeperiod=120)
data['atr2']=ATR(data,timeperiod=200)

# 初始部位
position = 0
signal = 0
# 開始回測
for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'high']
    c_close=data.loc[c_time,'close']
    c_ema=data.loc[c_time,'ema']
    c_atr1=data.loc[c_time,'atr1']
    c_atr2=data.loc[c_time,'atr2']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'open']

    # 進場程序
    if position ==0  :
        # 進場邏輯
        if c_close > c_ema*1.01 and c_atr1 > c_atr2 :
            position = 1
            signal = 3
        else:
            signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if  c_close < c_ema*0.995 :
            position = 0
            signal = 4
        else:
            signal = 2
result = get_company_name(prod)
strategy_name='ATR濾網交易策略'

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))


請輸入要查詢的股票代碼:2330


2024-01-25 09:15:32.112 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2330


ATR濾網交易策略 
 2330 台積電 
 2024-01-25 
維持空手


###**趨勢突破策略**

In [19]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data = getDataFM(prod,st,en)
data['Ceil'] = data.rolling(3)['High'].max().shift()

movestoploss = 0.05

position = 0
sugnal= 0

for i in range(data.shape[0]-1):
  c_time = data.index[i]
  c_high = data.loc[c_time,'High']
  c_close = data.loc[c_time,'Close']
  c_ceil = data.loc[c_time,'Ceil']

  n_time = data.index[i+1]
  n_open = data.loc[n_time,'Open']

  if position == 0 :
    if c_close > c_ceil :
      position = 1
      order_i = i
      order_time = n_time
      order_price = n_open
      stoploss = order_price * (1 - movestoploss)
      # print(c_time,'觸發進場訊號 隔日進場', order_time,'進場價', order_price, '進場', order_unit,'單位')
      signal = 3
    else:
      signal = 1
  elif position == 1 :
    stoploss = max(stoploss,c_close * (1 - movestoploss))
    if c_close < stoploss:
      position = 0
      cover_time = n_time
      cover_price = n_open
      signal = 4
    else:
      signal = 2
      # print(c_time,'觸發出場訊號 隔日出場', order_time, '出場價', order_price)
strategy_name='趨勢突破策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2401


2024-01-24 08:03:42.266 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2401


趨勢突破策略 
 2401 
 2024-01-24 
維持進場


###**跟著外資策略**

In [10]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndInstInvest_FM(prod,st,en)

# 計算 外資平均買賣
data['F_day']=data['外陸資買進股數(不含外資自營商)']-data['外陸資賣出股數(不含外資自營商)']
data['F_month']=(data['F_day']).rolling(40).sum()
data['F_mean']=(data['F_month']).rolling(20).mean()
# 初始部位
position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'High']
    c_close=data.loc[c_time,'Close']
    c_F_month=data.loc[c_time,'F_month']
    c_F_mean=data.loc[c_time,'F_mean']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'Open']

    # 進場程序
    if position == 0 :
        if c_F_month > c_F_mean :
            position = 1
            signal = 3
        else :
            signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_F_month < c_F_mean :
            position = 0
            signal = 4
        else:
            signal = 2

strategy_name='跟著外資策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2329


2024-01-25 03:48:07.926 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2329
2024-01-25 03:48:10.092 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockInstitutionalInvestorsBuySell, data_id: 2329


跟著外資策略 
 2329 
 2024-01-25 
維持空手


###**跟著大戶策略**

In [24]:
import datetime
import twstock
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2324


2024-01-25 13:38:45.788 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2324



 跟著大戶策略 
 2324 仁寶 
 2024-01-25 
進場訊號


###**反著散戶策略**

In [13]:
import datetime
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

# 30張持有以下稱為散戶
data['minority']=data['1']+data['2']+data['3']+data['4']
# 算出過去的平均
data['minority_ma']=data.rolling(8)['minority'].mean()
data['ma']=data.rolling(4)['close'].mean()
data['total_people_ma']=data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'high']
    c_close=data.loc[c_time,'close']
    c_ma=data.loc[c_time,'ma']
    c_minority=data.loc[c_time,'minority']
    c_minority_ma=data.loc[c_time,'minority_ma']
    c_total_people=data.loc[c_time,'total_people']
    c_total_people_ma=data.loc[c_time,'total_people_ma']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
        if c_close > c_ma and c_minority < c_minority_ma :
          position = 1
          signal = 3
        else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_minority > c_minority_ma:
            position = 0
            signal = 4
        else:
            signal = 2

strategy_name='反著散戶策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

請輸入要查詢的股票代碼:2331


2024-01-25 12:31:26.439 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2331



 反著散戶策略 
 2331 精英 
 2024-01-25 
維持空手


###**融資融券策略**

In [ ]:
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getFMPriceAndMarginTrade(prod,st,en)

# 融資融券餘額
data['mb']=data['MarginPurchaseTodayBalance'].astype(int)
data['ms']=data['ShortSaleTodayBalance'].astype(int)
data['mb_mean']=data['mb'].rolling(20).mean()
data['ms_mean']=data['ms'].rolling(20).mean()

# 初始部位
position=0
signal = 0
# 開始回測
for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'High']
    c_close=data.loc[c_time,'Close']
    c_mb=data.loc[c_time,'mb']
    c_mb_mean=data.loc[c_time,'mb_mean']
    c_ms=data.loc[c_time,'ms']
    c_ms_mean=data.loc[c_time,'ms_mean']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'Open']

    # 進場程序
    if position == 0 :
        if c_mb < c_mb_mean*0.99 and c_ms > c_ms_mean*1.01 :
            position = 1
            signal = 3
        else:
            signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_ms < c_ms_mean  :
            position = 0
            signal = 4
        else:
            signal = 2

strategy_name='融資融券策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

###**營收策略**

In [ ]:
# 載入必要的套件
import time,requests,os,datetime,random
from json import loads
import pandas as pd

# 切換工作路徑
# os.chdir('調整為自己的存檔路徑')

ym_list=[]
for y in range(2010,2024):
    for m in range(1,13):
        y=str(y)
        m=str(m).zfill(2)
        ym_list.append([y,m])

# 從 2010年開始爬蟲
first_write=True

for y,m in ym_list:
    ym=y+m
    try:
        print(ym)

        y = int(y)-1911
        m = int(m)

        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_%s_%s_0.html'%(y,m)

        # 下載該年月的網站，並用 pandas 轉換成 dataframe
        r = requests.get(url)
        r.encoding = 'cp950'
        # 解析成pd.Dataframe
        dfs = pd.read_html(r.text)

        # 取出欄位數量為11的表格
        data = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])

        # 調整欄位名稱
        if data.shape[1] == 10:
            data.columns=['公司代號','公司名稱','當月營收','上月營收','去年當月營收','上月比較增減(%)','去年同月增減(%)','當月累計營收','去年累計營收','前期比較增減(%)']
        else:
            data.columns=['公司代號','公司名稱','當月營收','上月營收','去年當月營收','上月比較增減(%)','去年同月增減(%)','當月累計營收','去年累計營收','前期比較增減(%)','備註']
        data['備註'] = ''
        data=data[data['公司代號'].str[-2:] != '合計']
        data['日期'] = ym+'10'

        if first_write:
            data.to_csv('月營收爬蟲資料.csv',encoding='cp950',index=False)
            first_write=False
        else:
            data.to_csv('月營收爬蟲資料.csv',encoding='cp950',mode='a',index=False,header=False)
    except:
        print(ym ,'爬蟲失敗')
        continue
    finally:
        time.sleep(random.randint(1,5))

In [ ]:
# 取得回測資料
prod=int(input("請輸入要查詢的股票代碼:"))
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')'
data=getTSEPriceAndRevenue(prod,st,en)

# 初始部位
position = 0
signal = 0
# 開始回測
for i in range(data.shape[0]-1):
    # 取得策略會應用到的變數
    c_time=data.index[i]
    c_high=data.loc[c_time,'High']
    c_close=data.loc[c_time,'Close']
    c_earn=data.loc[c_time,'前期比較增減(%)']
    # 取下一期資料做為進場資料
    n_time=data.index[i+1]
    n_open=data.loc[n_time,'Open']

    # 進場程序
    if position == 0 :
        if c_earn > 30 :
            position = 1
            order_i=i
            order_time=n_time
            order_price=n_open
            signal = 3
        else:
            signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
        if c_earn < 30 or i == data.shape[0]-2:
            position = 0
            cover_time=n_time
            cover_price=n_open
            signal = 4
        else:
            signal = 2

strategy_name='營收策略'
result = get_company_name(prod)

if signal==1:
    line_print('\n %s \n %s 【$s】 \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s 【%s】 \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s 【%s】 \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s 【%s】 \n %s \n出場訊號'%(strategy_name,prod,result,en))

###**我的持股**

---



####飛寶企業

In [22]:
import datetime
import twstock
# 取得回測資料
prod='4413'
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'
if signal==1:
    line_print('\n %s \n %s %s \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s %s \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s %s \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s %s \n %s \n出場訊號'%(strategy_name,prod,result,en))

2024-01-25 13:35:05.375 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 4413
2024-01-25 13:35:07.381 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockHoldingSharesPer, data_id: 4413



 跟著大戶策略 
 4413 飛寶 
 2024-01-25 
進場訊號


####精英

In [17]:
import datetime
import twstock
# 取得回測資料
prod="2331"
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'
if signal==1:
    line_print('\n %s \n %s %s \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s %s \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s %s \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s %s \n %s \n出場訊號'%(strategy_name,prod,result,en))

2024-01-25 13:32:37.816 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2331



 跟著大戶策略 
 2331 精英 
 2024-01-25 
維持空手


####志豐

In [18]:
import datetime
import twstock
# 取得回測資料
prod="3206"
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'
if signal==1:
    line_print('\n %s \n %s %s \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s %s \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s %s \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s %s \n %s \n出場訊號'%(strategy_name,prod,result,en))

2024-01-25 13:32:44.156 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 3206
2024-01-25 13:32:45.905 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockHoldingSharesPer, data_id: 3206



 跟著大戶策略 
 3206 志豐 
 2024-01-25 
維持空手


####康舒

In [19]:
import datetime
import twstock
# 取得回測資料
prod="6282"
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'
if signal==1:
    line_print('\n %s \n %s %s \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s %s \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s %s \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s %s \n %s \n出場訊號'%(strategy_name,prod,result,en))

2024-01-25 13:32:52.476 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 6282
2024-01-25 13:32:53.888 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockHoldingSharesPer, data_id: 6282



 跟著大戶策略 
 6282 康舒 
 2024-01-25 
維持空手


####敬鵬

In [20]:
import datetime
import twstock
# 取得回測資料
prod="2355"
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'
if signal==1:
    line_print('\n %s \n %s %s \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s %s \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s %s \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s %s \n %s \n出場訊號'%(strategy_name,prod,result,en))

2024-01-25 13:33:00.053 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2355
2024-01-25 13:33:01.430 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockHoldingSharesPer, data_id: 2355



 跟著大戶策略 
 2355 敬鵬 
 2024-01-25 
維持空手


####仁寶

In [23]:
import datetime
import twstock
# 取得回測資料
prod="2324"
st='2010-01-01'
en=datetime.datetime.now().strftime('%Y-%m-%d')
data=getPriceAndShareHolder(prod,st,en)

data['major'] = data['12'] + data['13'] + data['14'] + data['15'] #400張以上為大戶
data['major_ma'] = data.rolling(8)['major'].mean()
data['total_people_ma'] = data.rolling(8)['total_people'].mean()

position=0
# 當天訊號
signal=0

for i in range(data.shape[0]-1):
    c_time = data.index[i]
    c_high = data.loc[c_time,'high']
    c_close = data.loc[c_time,'close']
    c_major = data.loc[c_time,'major']
    c_major_ma = data.loc[c_time,'major_ma']
    c_total_people = data.loc[c_time,'total_people']
    c_total_people_ma = data.loc[c_time,'total_people_ma']

    n_time = data.index[i+1]
    n_open = data.loc[n_time,'open']

    # 進場程序
    if position == 0 :
      if c_major > c_major_ma and c_total_people > c_total_people_ma :
          position = 1
          signal = 3
      else :
          signal = 1
    # 出場程序
    elif position ==1 :
        # 出場邏輯
      if c_major < c_major_ma:
            position = 0
            signal = 4
      else:
            signal = 2

result = get_company_name(prod)
strategy_name='跟著大戶策略'
if signal==1:
    line_print('\n %s \n %s %s \n %s \n維持空手'%(strategy_name,prod,result,en))
elif signal==2:
    line_print('\n %s \n %s %s \n %s \n維持進場'%(strategy_name,prod,result,en))
elif signal==3:
    line_print('\n %s \n %s %s \n %s \n進場訊號'%(strategy_name,prod,result,en))
elif signal==4:
    line_print('\n %s \n %s %s \n %s \n出場訊號'%(strategy_name,prod,result,en))

2024-01-25 13:37:55.905 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2324
2024-01-25 13:37:58.025 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockHoldingSharesPer, data_id: 2324



 跟著大戶策略 
 2324 仁寶 
 2024-01-25 
進場訊號


In [6]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/usr/local/lib/python3.10/dist-packages/FinMind', '/usr/local/lib/python3.10/dist-packages/FinMind']
